# Action Castle - Complete Implementation

This notebook contains the complete implementation of Action Castle, a text adventure game based on the game from [Parsley](http://www.memento-mori.com/parsely-products/parsely-pdf) by Jared A. Sorensen.

## Game Overview
Action Castle is a text adventure where you play as a peasant who must navigate through a castle, solve puzzles, and ultimately become the reigning monarch by sitting on the throne.

## Win Condition
The game is won when any character sits on the throne (has the property `is_reigning`).

## Complete Win Route
1. Get the branch from the tree
2. Hit the guard with the branch (subdues guard)
3. Go east to feasting hall, get the candle
4. Light the candle
5. Go down to dungeon stairs, then down to dungeon
6. Read runes (banishes ghost, drops crown)
7. Get the crown
8. Go up to tower stairs, unlock door with key
9. Go up to tower, propose to princess (becomes royal)
10. Go to throne room, wear crown, sit on throne (wins!)


## Install Required Libraries


In [ ]:
%pip install -r requirements.txt


## Import Text Adventures Framework


In [ ]:
from text_adventure_games import (
    games, parsing, actions, things, blocks, viz
)


## Create All Locations

Action Castle has 13 locations that form the game world:


In [ ]:
# create all locs
cottage = things.Location(
    "Cottage",
    "You are standing in a small cottage."
)
garden_path = things.Location(
    "Garden Path",
    "You are standing on a lush garden path. There is a cottage here.",
)
fishing_pond = things.Location(
    "Fishing Pond",
    "You are at the edge of a small fishing pond."
)
winding_path = things.Location(
    "Winding Path",
    "You are walking along a winding path. There is a tall tree here.",
)
top_of_tree = things.Location(
    "Top of the Tall Tree",
    "You are the top of the tall tree."
)
drawbridge = things.Location(
    "Drawbridge",
    "You are standing on one side of a drawbridge leading to ACTION CASTLE.",
)
courtyard = things.Location(
    "Courtyard",
    "You are in the courtyard of ACTION CASTLE."
)
tower_stairs = things.Location(
    "Tower Stairs",
    "You are climbing the stairs to the tower. There is a locked door here.",
)
tower = things.Location(
    "Tower",
    "You are inside a tower."
)
dungeon_stairs = things.Location(
    "Dungeon Stairs",
    "You are climbing the stairs down to the dungeon."
)
dungeon = things.Location(
    "Dungeon",
    "You are in the dungeon. There is a spooky ghost here."
)
feasting_hall = things.Location(
    "Great Feasting Hall",
    "You stand inside the Great Feasting Hall."
)
throne_room = things.Location(
    "Throne Room",
    "This is the throne room of ACTION CASTLE."
)
death = things.Location(
    "The Afterlife",
    "You are dead. GAME OVER."
)
death.set_property("game_over", True)


## Connect Locations

Create the map by connecting locations with directional exits:


In [ ]:
# connect locs to create game map
cottage.add_connection("out", garden_path)
garden_path.add_connection("south", fishing_pond)
garden_path.add_connection("north", winding_path)
winding_path.add_connection("up", top_of_tree)
winding_path.add_connection("east", drawbridge)
top_of_tree.add_connection("jump", death)
drawbridge.add_connection("east", courtyard)
courtyard.add_connection("up", tower_stairs)
courtyard.add_connection("down", dungeon_stairs)
courtyard.add_connection("east", feasting_hall)
tower_stairs.add_connection("up", tower)
dungeon_stairs.add_connection("down", dungeon)
feasting_hall.add_connection("east", throne_room)


## Create Items

Items are objects that can be picked up, used, or interacted with:


In [ ]:
# gettable items
fishing_pole = things.Item(
    "pole",
    "a fishing pole",
    "A SIMPLE FISHING POLE.",
)
cottage.add_item(fishing_pole)

branch = things.Item(
    "branch",
    "a stout, dead branch",
    "IT LOOKS LIKE IT WOULD MAKE A GOOD CLUB.",
)
branch.set_property("is_weapon", True)
branch.set_property("is_fragile", True)
top_of_tree.add_item(branch)

candle = things.Item(
    "candle",
    "a strange candle",
    "THE CANDLE IS COVERED IN STRANGE RUNES.",
)
candle.set_property("is_lightable", True)
candle.set_property("is_lit", False)
candle.add_command_hint("light candle")
candle.add_command_hint("read runes")
feasting_hall.add_item(candle)

# sceney items (non-gettable)
pond = things.Item(
    "pond",
    "a small fishing pond",
    "THERE ARE FISH IN THE POND.",
)
pond.set_property("gettable", False)
pond.set_property("has_fish", True)
pond.add_command_hint("catch fish")
pond.add_command_hint("catch fish with pole")
fishing_pond.add_item(pond)

rosebush = things.Item(
    "rosebush",
    "a rosebush",
    "THE ROSEBUSH CONTAINS A SINGLE RED ROSE. IT IS BEAUTIFUL.",
)
rosebush.set_property("gettable", False)
rosebush.set_property("has_rose", True)
rosebush.add_command_hint("pick rose")
garden_path.add_item(rosebush)

throne = things.Item(
    "throne",
    "An ornate golden throne."
)
throne.set_property("gettable", False)
throne.add_command_hint("sit on throne")
throne_room.add_item(throne)

door = things.Item(
    "door",
    "a door",
    "THE DOOR IS SECURELY LOCKED."
)
door.set_property("gettable", False)
door.set_property("is_locked", True)
door.add_command_hint("unlock door")
tower_stairs.add_item(door)


## Create Characters

Characters include the player and NPCs (Non-Player Characters):


In [ ]:
# player
player = things.Character(
    name="The player",
    description="You are a simple peasant destined for greatness.",
    persona="I am on an adventure.",
)

# players lamp
lamp = things.Item("lamp", "a lamp", "A LAMP.")
lamp.set_property("is_lightable", True)
lamp.set_property("is_lit", False)
lamp.add_command_hint("light lamp")
player.add_to_inventory(lamp)

# npcs
troll = things.Character(
    name="troll",
    description="A mean troll",
    persona="I am hungry. The guard promised to feed me if I guard the drawbridge and keep people out of the castle.",
)
troll.set_property("is_hungry", True)
troll.set_property("character_type", "troll")
drawbridge.add_character(troll)

guard = things.Character(
    name="guard",
    description="A castle guard",
    persona="I am suspicious of anyone trying to enter the castle. I will prevent keep people from entering and learning the castle's dark secrets.",
)
guard.set_property("is_conscious", True)
guard.set_property("is_suspicious", True)
guard.set_property("character_type", "human")
courtyard.add_character(guard)

# guard has a key
key = things.Item("key", "a brass key", "THIS LOOKS USEFUL")
guard.add_to_inventory(key)

# guard has a sword
sword = things.Item("sword", "a short sword", "A SHARP SHORT SWORD.")
sword.set_property("is_weapon", True)
guard.add_to_inventory(sword)

princess = things.Character(
    name="princess",
    description="A princess who is beautiful and lonely. She awaits her non-gender-stereotypical soulmate.",
    persona="I am the princess. I am grieving my father's death. I feel alone.",
)
princess.set_property("is_royal", True)
princess.set_property("emotional_state", "sad and lonely")
princess.set_property("is_married", False)
princess.set_property("character_type", "human")
tower.add_character(princess)

ghost = things.Character(
    name="ghost",
    description="A ghost with bony, claw-like fingers and who is wearing a crown.",
    persona="I was murdered by the guard. I will haunt this castle until banished. If you linger before my apparition, I will plunge my ghostly hand inside you and stop your heart",
)
ghost.set_property("character_type", "ghost")
ghost.set_property("is_dead", True)
ghost.set_property("is_banished", False)
dungeon.add_character(ghost)

# ghosts crown
crown = things.Item("crown", "a crown", "A CROWN FIT FOR A KING.")
crown.add_command_hint("wear crown")
ghost.add_to_inventory(crown)


## Custom Actions

These are the special actions that make Action Castle unique:


In [ ]:
class Unlock_Door(actions.Action):
    ACTION_NAME = "unlock door"
    ACTION_DESCRIPTION = "Unlock a door with a key"
    ACTION_ALIASES = []
    
    def __init__(self, game, command):
        super().__init__(game)
        self.character = self.parser.get_character(command)
        self.door = self.parser.match_item(command, self.parser.get_items_in_scope(self.character))
        self.key = None
        
        # find a key in chars inventory
        for item_name, item in self.character.inventory.items():
            if "key" in item_name.lower():
                self.key = item
                break

    def check_preconditions(self) -> bool:
        """
        Preconditions:
        * There must be a door
        * The character must be at the same location as the door
        * The door must be locked
        * The character must have the key in their inventory
        """
        if not self.door:
            self.parser.fail("There is no door here to unlock.")
            return False
            
        if not self.character.location.here(self.door):
            self.parser.fail("You must be at the door to unlock it.")
            return False
            
        if not self.door.get_property("is_locked"):
            self.parser.fail("The door is already unlocked.")
            return False
            
        if not self.key:
            self.parser.fail("You need a key to unlock the door.")
            return False
            
        return True

    def apply_effects(self):
        """
        Effects:
        * Unlocks the door
        """
        self.door.set_property("is_locked", False)
        self.parser.ok(f"You unlock the door with the {self.key.name}.")


In [ ]:
class Read_Runes(actions.Action):
    """
    Reading the runes on the candle with strange runes on it will banish the
    ghost from the dungeon, and cause it to drop the crown.
    """
    ACTION_NAME = "read runes"
    ACTION_DESCRIPTION = "Read runes off of the candle"
    ACTION_ALIASES = []

    def __init__(self, game, command):
        super().__init__(game)
        self.character = self.parser.get_character(command)
        self.candle = None
        self.ghost = None
        
        # Find a candle in the character's inventory
        for item_name, item in self.character.inventory.items():
            if "candle" in item_name.lower():
                self.candle = item
                break
                
        # Find the ghost in the current location
        for char_name, char in self.character.location.characters.items():
            if char.get_property("character_type") == "ghost":
                self.ghost = char
                break

    def check_preconditions(self) -> bool:
        """
        Preconditions:
        * There must be a candle with strange runes on it
        * The character must have the candle in their inventory
        * the ghost must be in this location
        * The candle must be lit
        """
        if not self.candle:
            self.parser.fail("You need a candle with runes to read.")
            return False
            
        if not self.candle.get_property("is_lit"):
            self.parser.fail("The candle must be lit to read the runes.")
            return False
            
        if not self.ghost:
            self.parser.fail("There is no ghost here to banish.")
            return False
            
        if self.ghost.get_property("is_banished"):
            self.parser.fail("The ghost has already been banished.")
            return False
            
        return True

    def apply_effects(self):
        """
        Effects:
        * Banishes the ghost, causing it to drop its inventory.
        """
        # Banish the ghost
        self.ghost.set_property("is_banished", True)
        
        # Move ghost to a different location (or remove from current location)
        self.character.location.remove_character(self.ghost)
        
        # Drop the ghost's inventory
        for item_name, item in list(self.ghost.inventory.items()):
            self.ghost.remove_from_inventory(item_name)
            self.character.location.add_item(item)
            
        self.parser.ok("You read the ancient runes on the candle. The ghost shrieks and vanishes, leaving behind its crown!")


In [ ]:
class Propose(actions.Action):
    """
    Mawwige is whut bwings us togevveh today.
    """
    ACTION_NAME = "propose marriage"
    ACTION_DESCRIPTION = "Propose marriage to someone"
    ACTION_ALIASES = []

    def __init__(self, game, command):
        super().__init__(game)
        self.proposer = self.parser.get_character(command)
        self.propositioned = None
        
        # Find another character in the same location
        for char_name, char in self.proposer.location.characters.items():
            if char != self.proposer:
                self.propositioned = char
                break

    def check_preconditions(self) -> bool:
        """
        Preconditions:
        * The two characters must be in the same place
        * Neither can be married yet
        * Both must be happy
        """
        if not self.propositioned:
            self.parser.fail("There is no one here to propose to.")
            return False
            
        if self.proposer.get_property("is_married"):
            self.parser.fail("You are already married.")
            return False
            
        if self.propositioned.get_property("is_married"):
            self.parser.fail(f"The {self.propositioned.name} is already married.")
            return False

        # assume consensual marriage (obvs)
        
        return True

    def apply_effects(self):
        """
        Effects:
        * They said "Yes!"
        * They are married.
        * If one is a royal, they are now both royals
        """
        # mark both as married
        self.proposer.set_property("is_married", True)
        self.propositioned.set_property("is_married", True)
        
        # if one is royal, make both royal
        if self.propositioned.get_property("is_royal"):
            self.proposer.set_property("is_royal", True)
            self.parser.ok(f"You propose to the {self.propositioned.name}. She says 'Yes!' You are now married and have become royal!")
        else:
            self.parser.ok(f"You propose to the {self.propositioned.name}. They say 'Yes!' You are now married!")


In [ ]:
class Wear_Crown(actions.Action):
    ACTION_NAME = "wear crown"
    ACTION_DESCRIPTION = "Put a crown in your inventory atop your head"
    ACTION_ALIASES = []
    
    def __init__(self, game, command):
        super().__init__(game)
        self.character = self.parser.get_character(command)
        self.crown = None
        
        # find a crown in the chars inventory
        for item_name, item in self.character.inventory.items():
            if "crown" in item_name.lower():
                self.crown = item
                break

    def check_preconditions(self) -> bool:
        """
        Preconditions:
        * The crown must be in the character's inventory
        * The character must be a royal
        """
        if not self.crown:
            self.parser.fail("You don't have a crown to wear.")
            return False
            
        if not self.character.get_property("is_royal"):
            self.parser.fail("Only royalty can wear a crown.")
            return False
            
        return True

    def apply_effects(self):
        """
        The character is crowned.
        """
        self.character.set_property("is_crowned", True)
        self.parser.ok("You place the crown upon your head. You feel the weight of royal responsibility.")


In [ ]:
class Sit_On_Throne(actions.Action):
    ACTION_NAME = "sit on throne"
    ACTION_DESCRIPTION = "Sit on the throne, if you are royalty"
    ACTION_ALIASES = []
    
    def __init__(self, game, command):
        super().__init__(game)
        self.character = self.parser.get_character(command)
        self.throne = self.parser.match_item(command, self.parser.get_items_in_scope(self.character))

    def check_preconditions(self) -> bool:
        """
        Preconditions:
        * The character must be in same location as the throne
        * The character must be a royal
        * The character must be wearing a crown
        """
        if not self.throne:
            self.parser.fail("There is no throne here to sit on.")
            return False
            
        if not self.character.get_property("is_royal"):
            self.parser.fail("Only royalty can sit on the throne.")
            return False
            
        if not self.character.get_property("is_crowned"):
            self.parser.fail("You must be wearing a crown to sit on the throne.")
            return False
            
        return True

    def apply_effects(self):
        """
        The character becomes the reigning monarch.
        """
        self.character.set_property("is_reigning", True)
        self.parser.ok("You sit upon the throne. You are now the reigning monarch of Action Castle!")


## Custom Blocks

Blocks are puzzles that prevent progress until certain conditions are met:


In [ ]:
# sample block, troll block (already implemented)
class Troll_Block(blocks.Block):
    """
    Blocks progress in this direction until the troll is no longer hungry, or
    leaves, or is unconscious, or dead.
    """

    def __init__(self, location: things.Location, troll: things.Character):
        super().__init__(
            "A troll blocks your way", "A hungry troll blocks your way"
        )
        self.location = location
        self.troll = troll

    def is_blocked(self) -> bool:
        # conditions of block:
        # * there is a troll here
        # * troll is alive and conscious
        # * troll is still hungry
        if self.troll:
            if not self.location.here(self.troll):
                return False
            if self.troll.get_property("is_dead"):
                return False
            if self.troll.get_property("is_unconscious"):
                return False
            if self.troll.get_property("is_hungry"):
                return True
        return False


In [ ]:
class Guard_Block(blocks.Block):
    """
    Blocks progress in this direction until the guard is no longer suspicious, or
    leaves, or is unconscious, or dead.
    """

    def __init__(self, location: things.Location, guard: things.Character):
        super().__init__(
            "A guard blocks your way", "The guard refuses to let you pass."
        )
        self.location = location
        self.guard = guard

    def is_blocked(self) -> bool:
        # conditions of block:
        # * there is a guard here
        # * guard is alive and conscious
        # * guard is suspicious
        if self.guard:
            if not self.location.here(self.guard):
                return False
            if self.guard.get_property("is_dead"):
                return False
            if self.guard.get_property("is_unconscious"):
                return False
            if self.guard.get_property("is_suspicious"):
                return True
        return False


In [ ]:
class Darkness(blocks.Block):
    """
    Blocks progress in this direction unless the character has something that lights the way.
    """

    def __init__(self, location: things.Location):
        super().__init__("Darkness blocks your way", "It's too dark to go that way.")
        self.location = location

    def is_blocked(self) -> bool:
        # conditions of block:
        # * the location is dark
        # * unblocked if any char at the loc is carrying a lit item (like a lamp or candle)
        for char_name, character in self.location.characters.items():
            for item_name, item in character.inventory.items():
                if item.get_property("is_lit"):
                    return False
        return True


In [ ]:
class Door_Block(blocks.Block):
    """
    Blocks progress in this direction until the character unlocks the door.
    """

    def __init__(self, location: things.Location, door: things.Item):
        super().__init__("A locked door blocks your way", "The door ahead is locked.")
        self.location = location
        self.door = door

    def is_blocked(self) -> bool:
        # conditions of block:
        # * door is locked
        return self.door.get_property("is_locked") is True


## Add Blocks to Locations

Add the required blocks to prevent progress until puzzles are solved:


In [ ]:
# add blocks to locations:
# * courtyard, guard prevents you from going East
# * dungeon_stairs, darkness prevents you from going Down
# * tower stairs, locked door prevents you from going Up

# guard block for courtyard east, to feasting hall
guard_block = Guard_Block(courtyard, guard)
courtyard.add_block("east", guard_block)

# darkness block for dungeon_stairs down, to dungeon
darkness_block = Darkness(dungeon_stairs)
dungeon_stairs.add_block("down", darkness_block)

# locked door block for tower_stairs up, to tower
locked_door_block = Door_Block(tower_stairs, door)
tower_stairs.add_block("up", locked_door_block)

# existing troll block
troll_block = Troll_Block(drawbridge, troll)
drawbridge.add_block("east", troll_block)


## Create the Game


In [ ]:
class ActionCastle(games.Game):
    def __init__(
        self, start_at: things.Location, player: things.Character, characters=None,
        custom_actions=None
    ):
        super().__init__(start_at, player, characters=characters, custom_actions=custom_actions)

    def is_won(self) -> bool:
        """ 
        Checks whether the game has been won. For Action Castle, the game is won
        once any character is sitting on the throne (has the property is_reigning).
        """
        for name, character in self.characters.items():
            if character.get_property("is_reigning"):
                msg = "{name} now reigns in ACTION CASTLE! {name} has won the game!"
                self.parser.ok(msg.format(name=character.name.title()))
                return True
        return False


In [ ]:
# create the game with all chars and custom actions
characters = [troll, guard, princess, ghost]
custom_actions = [Unlock_Door, Read_Runes, Propose, Wear_Crown, Sit_On_Throne]

# game
game = ActionCastle(cottage, player, characters=characters, custom_actions=custom_actions)

print("Action Castle game created successfully!")
print("All 13 locations created and connected!")
print("All items and characters placed!")
print("All custom actions implemented!")
print("All blocks added to prevent progress!")
print("Game ready to play!")


## How to Play Action Castle

### Available Commands:
- **Basic Commands**: `look`, `inventory`, `take <item>`, `drop <item>`, `go <direction>`
- **Special Commands**: `light <item>`, `attack <character>`, `give <item> to <character>`
- **Custom Actions**: `unlock door`, `read runes`, `propose`, `wear crown`, `sit on throne`

### Complete Win Route:
1. **Get the branch** from the tree (`take branch`)
2. **Hit the guard** with the branch (`attack guard`) - this subdues the guard
3. **Go east** to feasting hall, **get the candle** (`take candle`)
4. **Light the candle** (`light candle`)
5. **Go down** to dungeon stairs, then **down to dungeon**
6. **Read runes** (`read runes`) - banishes ghost, drops crown
7. **Get the crown** (`take crown`)
8. **Go up** to tower stairs, **unlock door** (`unlock door`)
9. **Go up** to tower, **propose to princess** (`propose`) - becomes royal
10. **Go to throne room**, **wear crown** (`wear crown`), **sit on throne** (`sit on throne`) - wins!

### Start the Game:


In [ ]:
# Start the game!
game.game_loop()
